# Projet Final de Machine Learning

Import des données

In [4]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit
import seaborn as sns
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=d126d553f610247e0be98cf7b0f9f96675094b99e512ad3f1c1c6ddc32f593b7
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


Import du fichier csv et création du dataframe

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('sample_data/attrition_log_reg').getOrCreate()
# File location and type
file_location = "/content/sample_data/HR-Employee-Attrition.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

DataFrame[Age: int, Attrition: string, BusinessTravel: string, DailyRate: int, Department: string, DistanceFromHome: int, Education: int, EducationField: string, EmployeeCount: int, EmployeeNumber: int, EnvironmentSatisfaction: int, Gender: string, HourlyRate: int, JobInvolvement: int, JobLevel: int, JobRole: string, JobSatisfaction: int, MaritalStatus: string, MonthlyIncome: int, MonthlyRate: int, NumCompaniesWorked: int, Over18: string, OverTime: string, PercentSalaryHike: int, PerformanceRating: int, RelationshipSatisfaction: int, StandardHours: int, StockOptionLevel: int, TotalWorkingYears: int, TrainingTimesLastYear: int, WorkLifeBalance: int, YearsAtCompany: int, YearsInCurrentRole: int, YearsSinceLastPromotion: int, YearsWithCurrManager: int]

In [15]:

data.cache()  # Cache data for faster reuse
data.count()

1470

Notre dataframe contient 1470 exemples d'employés.

Vérifions les colonnes qu'il contient

In [16]:
data.columns

['Age',
 'Attrition',
 'BusinessTravel',
 'DailyRate',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeCount',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'HourlyRate',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'MonthlyRate',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

Ensuite, explorons les données pour observer leurs distributions

In [17]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Age,1470,36.923809523809524,9.135373489136729,18,60
Attrition,1470,None,None,No,Yes
BusinessTravel,1470,None,None,Non-Travel,Travel_Rarely
DailyRate,1470,802.4857142857143,403.50909994352804,102,1499
Department,1470,None,None,Human Resources,Sales
DistanceFromHome,1470,9.19251700680272,8.10686443566608,1,29
Education,1470,2.912925170068027,1.0241649445978718,1,5
EducationField,1470,None,None,Human Resources,Technical Degree
EmployeeCount,1470,1.0,0.0,1,1


In [18]:
data = data.dropna() 
data.count()

1470

Nous avons donc pas de données nulles

In [19]:
data.dtypes

[('Age', 'int'),
 ('Attrition', 'string'),
 ('BusinessTravel', 'string'),
 ('DailyRate', 'int'),
 ('Department', 'string'),
 ('DistanceFromHome', 'int'),
 ('Education', 'int'),
 ('EducationField', 'string'),
 ('EmployeeCount', 'int'),
 ('EmployeeNumber', 'int'),
 ('EnvironmentSatisfaction', 'int'),
 ('Gender', 'string'),
 ('HourlyRate', 'int'),
 ('JobInvolvement', 'int'),
 ('JobLevel', 'int'),
 ('JobRole', 'string'),
 ('JobSatisfaction', 'int'),
 ('MaritalStatus', 'string'),
 ('MonthlyIncome', 'int'),
 ('MonthlyRate', 'int'),
 ('NumCompaniesWorked', 'int'),
 ('Over18', 'string'),
 ('OverTime', 'string'),
 ('PercentSalaryHike', 'int'),
 ('PerformanceRating', 'int'),
 ('RelationshipSatisfaction', 'int'),
 ('StandardHours', 'int'),
 ('StockOptionLevel', 'int'),
 ('TotalWorkingYears', 'int'),
 ('TrainingTimesLastYear', 'int'),
 ('WorkLifeBalance', 'int'),
 ('YearsAtCompany', 'int'),
 ('YearsInCurrentRole', 'int'),
 ('YearsSinceLastPromotion', 'int'),
 ('YearsWithCurrManager', 'int')]

trouver toutes les colonnes avec des valeurs nulles

In [20]:
for col in data.columns:
    print(col, data.filter(data[col].isNull()).count())

Age 0
Attrition 0
BusinessTravel 0
DailyRate 0
Department 0
DistanceFromHome 0
Education 0
EducationField 0
EmployeeCount 0
EmployeeNumber 0
EnvironmentSatisfaction 0
Gender 0
HourlyRate 0
JobInvolvement 0
JobLevel 0
JobRole 0
JobSatisfaction 0
MaritalStatus 0
MonthlyIncome 0
MonthlyRate 0
NumCompaniesWorked 0
Over18 0
OverTime 0
PercentSalaryHike 0
PerformanceRating 0
RelationshipSatisfaction 0
StandardHours 0
StockOptionLevel 0
TotalWorkingYears 0
TrainingTimesLastYear 0
WorkLifeBalance 0
YearsAtCompany 0
YearsInCurrentRole 0
YearsSinceLastPromotion 0
YearsWithCurrManager 0


In [21]:
data_features= data.select("Age", "MonthlyIncome", "JobRole", "TotalWorkingYears", "Attrition")

In [29]:
data_features.show()

+---+-------------+--------------------+-----------------+---------+
|Age|MonthlyIncome|             JobRole|TotalWorkingYears|Attrition|
+---+-------------+--------------------+-----------------+---------+
| 41|         5993|     Sales Executive|                8|      Yes|
| 49|         5130|  Research Scientist|               10|       No|
| 37|         2090|Laboratory Techni...|                7|      Yes|
| 33|         2909|  Research Scientist|                8|       No|
| 27|         3468|Laboratory Techni...|                6|       No|
| 32|         3068|Laboratory Techni...|                8|       No|
| 59|         2670|Laboratory Techni...|               12|       No|
| 30|         2693|Laboratory Techni...|                1|       No|
| 38|         9526|Manufacturing Dir...|               10|       No|
| 36|         5237|Healthcare Repres...|               17|       No|
| 35|         2426|Laboratory Techni...|                6|       No|
| 29|         4193|Laboratory Tech

In [ ]:
#correlation_matrix = data.select("*").toPandas().corr()
#display(correlation_matrix)

C'est important de connaitre les types des colonnes car le modèle ne prend que des valeurs numériques, ce que nous allons vectoriser toutes les colonnes de type string

In [ ]:
data.dtypes

[('Age', 'int'),
 ('Attrition', 'string'),
 ('BusinessTravel', 'string'),
 ('DailyRate', 'int'),
 ('Department', 'string'),
 ('DistanceFromHome', 'int'),
 ('Education', 'int'),
 ('EducationField', 'string'),
 ('EmployeeCount', 'int'),
 ('EmployeeNumber', 'int'),
 ('EnvironmentSatisfaction', 'int'),
 ('Gender', 'string'),
 ('HourlyRate', 'int'),
 ('JobInvolvement', 'int'),
 ('JobLevel', 'int'),
 ('JobRole', 'string'),
 ('JobSatisfaction', 'int'),
 ('MaritalStatus', 'string'),
 ('MonthlyIncome', 'int'),
 ('MonthlyRate', 'int'),
 ('NumCompaniesWorked', 'int'),
 ('Over18', 'string'),
 ('OverTime', 'string'),
 ('PercentSalaryHike', 'int'),
 ('PerformanceRating', 'int'),
 ('RelationshipSatisfaction', 'int'),
 ('StandardHours', 'int'),
 ('StockOptionLevel', 'int'),
 ('TotalWorkingYears', 'int'),
 ('TrainingTimesLastYear', 'int'),
 ('WorkLifeBalance', 'int'),
 ('YearsAtCompany', 'int'),
 ('YearsInCurrentRole', 'int'),
 ('YearsSinceLastPromotion', 'int'),
 ('YearsWithCurrManager', 'int')]

Nous allons ensuite vectoriser les colonnes et créer un pipeline pour enregistrer tous ses traitements

In [23]:
from pyspark.ml import Pipeline
 
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["JobRole", "BusinessTravel", "MaritalStatus", "EducationField", "Gender", "Department", "OverTime"]
#categoricalColumns = ["JobRole"]
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
print(stages)

[StringIndexer_cbcadf9d83a3, OneHotEncoder_19bf06a94288, StringIndexer_00dbd87c7452, OneHotEncoder_87623686cd93, StringIndexer_a53b6afad3c4, OneHotEncoder_8643a06db68c, StringIndexer_542949b82cd6, OneHotEncoder_29f59ff70b7f, StringIndexer_677e54091e13, OneHotEncoder_f0d426102cd0, StringIndexer_9676e4b230cd, OneHotEncoder_e3cb4758f723, StringIndexer_86491c29589c, OneHotEncoder_3934b8caf38c]


In [24]:
label_stringIdx = StringIndexer(inputCol = 'Attrition', outputCol = 'label')
stages += [label_stringIdx]

In [25]:
numericCols = ["Age", "DailyRate", "DistanceFromHome", "Education", "EmployeeCount", "EmployeeNumber", "EnvironmentSatisfaction", "HourlyRate", "JobInvolvement", "JobLevel", "JobSatisfaction", "MonthlyIncome", "NumCompaniesWorked", "TotalWorkingYears", "PercentSalaryHike", "PerformanceRating", "RelationshipSatisfaction", "StandardHours", "StockOptionLevel", "TrainingTimesLastYear", "WorkLifeBalance", "YearsAtCompany", "YearsInCurrentRole", "YearsSinceLastPromotion", "YearsWithCurrManager"]
#numericCols = ["Age", "MonthlyIncome", "TotalWorkingYears"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [26]:
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(data)
df = pipelineModel.transform(data)
selectedcols = ["label", "features"] + numericCols
df = df.select(selectedcols)
df.show(3)

+-----+--------------------+---+---------+----------------+---------+-------------+--------------+-----------------------+----------+--------------+--------+---------------+-------------+------------------+-----------------+-----------------+-----------------+------------------------+-------------+----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|label|            features|Age|DailyRate|DistanceFromHome|Education|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|HourlyRate|JobInvolvement|JobLevel|JobSatisfaction|MonthlyIncome|NumCompaniesWorked|TotalWorkingYears|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany|YearsInCurrentRole|YearsSinceLastPromotion|YearsWithCurrManager|
+-----+--------------------+---+---------+----------------+---------+-------------+--------------+-----------------------+----------

In [30]:
df.show()

+-----+--------------------+---+---------+----------------+---------+-------------+--------------+-----------------------+----------+--------------+--------+---------------+-------------+------------------+-----------------+-----------------+-----------------+------------------------+-------------+----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|label|            features|Age|DailyRate|DistanceFromHome|Education|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|HourlyRate|JobInvolvement|JobLevel|JobSatisfaction|MonthlyIncome|NumCompaniesWorked|TotalWorkingYears|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TrainingTimesLastYear|WorkLifeBalance|YearsAtCompany|YearsInCurrentRole|YearsSinceLastPromotion|YearsWithCurrManager|
+-----+--------------------+---+---------+----------------+---------+-------------+--------------+-----------------------+----------

Nous allons spliter notre dataframe avec 70% des données pour l'entrainement et 30% pour le test.

In [28]:
train, test = df.randomSplit([0.7, 0.3], seed=100)
print(train.count())
print(test.count())

1041
429


Nous allons choisir un des algorithmes de classification les plus connus comme modèle : la LogisticRegression

In [33]:
# import du modèle
from pyspark.ml.classification import LogisticRegression

#Implémentation
lr = LogisticRegression(labelCol = 'label', featuresCol = 'features', maxIter=10)

# entrainement du modèle
lrModel = lr.fit(train)

In [37]:
# prédiction
predictions = lrModel.transform(test)
predictions.take(1)

[Row(label=0.0, features=SparseVector(46, {0: 1.0, 8: 1.0, 10: 1.0, 13: 1.0, 19: 1.0, 20: 1.0, 21: 35.0, 22: 1219.0, 23: 18.0, 24: 3.0, 25: 1.0, 26: 975.0, 27: 3.0, 28: 86.0, 29: 3.0, 30: 2.0, 31: 3.0, 32: 4601.0, 33: 1.0, 34: 5.0, 35: 16.0, 36: 3.0, 37: 2.0, 38: 80.0, 40: 3.0, 41: 3.0, 42: 5.0, 43: 2.0, 44: 1.0}), Age=35, DailyRate=1219, DistanceFromHome=18, Education=3, EmployeeCount=1, EmployeeNumber=975, EnvironmentSatisfaction=3, HourlyRate=86, JobInvolvement=3, JobLevel=2, JobSatisfaction=3, MonthlyIncome=4601, NumCompaniesWorked=1, TotalWorkingYears=5, PercentSalaryHike=16, PerformanceRating=3, RelationshipSatisfaction=2, StandardHours=80, StockOptionLevel=0, TrainingTimesLastYear=3, WorkLifeBalance=3, YearsAtCompany=5, YearsInCurrentRole=2, YearsSinceLastPromotion=1, YearsWithCurrManager=0, rawPrediction=DenseVector([3.1311, -3.1311]), probability=DenseVector([0.9582, 0.0418]), prediction=0.0)]

In [35]:
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Age: integer (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- TotalWorkingYears: integer (nullable = true)
 |-- PercentSalaryHike: integer (nullable = true)
 |-- PerformanceRating: integer (nullable = true)
 |-- RelationshipSatisfaction: integer (nullable = true)
 |-- StandardHours: integer (nullable = true)
 |-- StockOptionLevel: integer (nullable = true)
 |-- TrainingTimesLas

Sélectionnons quelques colonnes pour voir les prédictions faites

In [38]:
selected = predictions.select("label", "age", "MonthlyIncome", "TotalWorkingYears", "prediction", "probability")
display(selected)

DataFrame[label: double, age: int, MonthlyIncome: int, TotalWorkingYears: int, prediction: double, probability: vector]

In [39]:
selected.show(4)

+-----+---+-------------+-----------------+----------+--------------------+
|label|age|MonthlyIncome|TotalWorkingYears|prediction|         probability|
+-----+---+-------------+-----------------+----------+--------------------+
|  0.0| 35|         4601|                5|       0.0|[0.95815801141235...|
|  0.0| 53|        10934|               35|       0.0|[0.94176420761724...|
|  0.0| 27|         5769|                6|       0.0|[0.71427175616604...|
|  0.0| 32|         5228|               13|       1.0|[0.34129765185840...|
+-----+---+-------------+-----------------+----------+--------------------+
only showing top 4 rows



Maintenant, nous allons évaluer notre modèle en utilisant BinaryClassificationEvaluator

In [40]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print('Area Under ROC', evaluator.evaluate(predictions))

Area Under ROC 0.8233575423658072


In [41]:
evaluator.getMetricName()

'areaUnderROC'

Essayons la crossvalidation

In [42]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [43]:
l=[]
for numFold in range(1,6):
    cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=numFold)
    l.append(cv)
print(l)


# Run cross validations
#cvModel1 = l[1].fit(train)
cvModel2 = l[2].fit(train)
#cvModel3 = l[3].fit(train)


[CrossValidator_d9628efe7b52, CrossValidator_a35c80c511d5, CrossValidator_b57b42c0887f, CrossValidator_c169e6a5083a, CrossValidator_fc5e062368ff]


In [44]:
predictions = cvModel2.transform(test)
print('Area Under ROC', evaluator.evaluate(predictions))

Area Under ROC 0.8250688705234156


Nous obtenons donc un score de 82% pour notre modèle de LogisticRegression. Cependant, il existe des modèles beaucoup plus puissants comme Random Forest qui rassemble 2 algorithmes : les arbres de décision et l'ensemble learning pour choisir le meilleur arbre. Nous allons tester cela.

Random Forest

In [45]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Age: integer (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- TotalWorkingYears: integer (nullable = true)
 |-- PercentSalaryHike: integer (nullable = true)
 |-- PerformanceRating: integer (nullable = true)
 |-- RelationshipSatisfaction: integer (nullable = true)
 |-- StandardHours: integer (nullable = true)
 |-- StockOptionLevel: integer (nullable = true)
 |-- TrainingTimesLas

In [46]:
selected = predictions.select("label", "prediction", "probability", "Age", "MonthlyIncome", "TotalWorkingYears")
display(selected)

DataFrame[label: double, prediction: double, probability: vector, Age: int, MonthlyIncome: int, TotalWorkingYears: int]

In [47]:
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.781576091493447

In [48]:
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(train)

In [49]:
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

0.7677393772435085

In [50]:
selected = predictions.select("label", "prediction", "probability", "Age", "MonthlyIncome", "TotalWorkingYears")
display(selected)

DataFrame[label: double, prediction: double, probability: vector, Age: int, MonthlyIncome: int, TotalWorkingYears: int]

In [51]:
bestModel = cvModel.bestModel
final_predictions = bestModel.transform(df)
evaluator.evaluate(final_predictions)
#bestModel

0.915084815944086

Nous obtenons un score de 92% avec notre bestModel qui est beaucoup meilleur que le modèle précédent. Nous allons donc conserver celui là.

In [52]:
# Enregistrons notre modèle
bestModel.save("best_Model")

In [55]:
# install du module pytest-shutil pour zipper notre modèle
!pip install pytest-shutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import shutil
shutil.make_archive('best_Model', 'zip', '/content/best_Model')

'/content/best_Model.zip'

Nous enregistrons notre modèle pour le déployer et le tester via un api.